In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import dataretrieval as nwis
from hydroeval import evaluator, nse, kge, pbias
from tqdm import tqdm
import datetime as dt

In [110]:
i = 163
pnwNP = pd.read_csv("../data/pnwNP_StatsLocations_SS.csv")
df = pd.read_csv('../data/pnwNP_modeledData/'+str(pnwNP['gage'][i])+".csv")
df['time'] = pd.to_datetime(df['time'])
df['water_year'] = df.time.dt.year.where(df.time.dt.month < 10, df.time.dt.year + 1)
df

,Unnamed: 0,streamflow_VIC,time,streamflow_PRMS,streamflow_NWM2d0,streamflow_NWM2d1,streamflow_NWIS,gage,water_year
0,0,2.881029,1950-01-01,4.524933,NaN,NaN,NaN,12447383,1950
1,1,4.845531,1950-01-02,7.295784,NaN,NaN,NaN,12447383,1950
2,2,5.303875,1950-01-03,7.620400,NaN,NaN,NaN,12447383,1950
3,3,5.325573,1950-01-04,7.322917,NaN,NaN,NaN,12447383,1950
4,4,5.266046,1950-01-05,6.951945,NaN,NaN,NaN,12447383,1950
...,...,...,...,...,...,...,...,...,...
25928,25928,NaN,2020-12-27,NaN,NaN,8.955000,3.256437,12447383,2021
25929,25929,NaN,2020-12-28,NaN,NaN,8.840833,3.256437,12447383,2021
25930,25930,NaN,2020-12-29,NaN,NaN,8.727083,3.171487,12447383,2021
25931,25931,NaN,2020-12-30,NaN,NaN,8.615416,3.086536,12447383,2021


In [40]:
pnwNP

,Unnamed: 0,gage,dec_lat,dec_long,numZero,numOne,startDate,endDate,comid,numObs,propZero,propOne,minProb_SS,maxProb_SS,meanProb_SS,countProb_SS,medianProb_SS
0,0,10361700,41.722116,-119.373258,2797,3228,1963-10-02,1972-09-29,24061841.0,3285,0.851446,0.982648,0.112000,0.112769,0.112128,6,0.112000
1,1,10366000,42.072111,-119.962725,0,61,1950-01-02,1991-10-17,24042897.0,15263,0.000000,0.003997,NaN,NaN,NaN,0,NaN
2,2,10378500,42.424883,-119.923285,0,2405,1950-01-02,1991-09-30,24043081.0,15246,0.000000,0.157746,0.607846,0.607846,0.607846,3,0.607846
3,3,10387500,42.999869,-120.751093,8177,8544,1951-06-19,1991-09-29,NaN,14712,0.555805,0.580750,0.074308,0.096923,0.085831,10,0.086308
4,4,10389500,43.110973,-121.069165,3131,4183,1965-10-02,1991-09-29,24026288.0,9493,0.329822,0.440640,0.043231,0.590615,0.434198,7,0.590615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613,613,14375400,42.031501,-123.737848,0,10,1985-10-02,1991-09-30,23945187.0,2189,0.000000,0.004568,NaN,NaN,NaN,0,NaN
614,614,433247110491701,43.546317,-110.822154,53,53,2008-07-23,2010-11-07,24433125.0,837,0.063321,0.063321,0.734923,0.737846,0.735569,5,0.735077
615,615,452755122234100,45.465120,-122.395923,4,1007,2013-01-02,2019-10-14,23815490.0,2476,0.001616,0.406704,0.514308,0.618462,0.584365,8,0.602538
616,616,452807122215001,45.468667,-122.363972,1,817,2015-10-29,2018-10-17,23815474.0,1084,0.000923,0.753690,0.383538,0.387385,0.385923,6,0.386154


In [48]:
tt = df[(df['streamflow_NWIS'].notna())&(df['streamflow_VIC'].notna())][['streamflow_NWIS','streamflow_VIC']].reset_index(drop=True)
evaluator(pbias, tt['streamflow_VIC'], tt['streamflow_NWIS'], axis=1)

array([-77.99058338])

In [43]:
tt

,streamflow_NWIS,streamflow_VIC
0,22.228725,11.786988
1,28.316847,14.077902
2,38.794080,16.728704
3,44.740618,16.679167
4,46.156460,14.708328
...,...,...
7190,0.000000,8.295341
7191,0.000000,8.183731
7192,0.000000,8.082645
7193,0.000000,7.983723


In [45]:
tt

,streamflow_NWIS,streamflow_VIC
0,22.228725,11.786988
1,28.316847,14.077902
2,38.794080,16.728704
3,44.740618,16.679167
4,46.156460,14.708328
...,...,...
7190,0.000000,8.295341
7191,0.000000,8.183731
7192,0.000000,8.082645
7193,0.000000,7.983723


## Inverse transform

In [101]:
tt = df[df['streamflow_NWIS'].notna()]

In [117]:
tt  =df

In [119]:
tt['streamflow_NWM2d0'] = tt['streamflow_NWM2d0'].notna().values() + eta

TypeError: 'numpy.ndarray' object is not callable

In [121]:
tt.loc[tt['streamflow_NWM2d0'].notna(),'streamflow_NWM2d0'] = tt.loc[tt['streamflow_NWM2d0'].notna(),'streamflow_NWM2d0']+ eta

In [129]:
tt.notna()

,Unnamed: 0,streamflow_VIC,time,streamflow_PRMS,streamflow_NWM2d0,streamflow_NWM2d1,streamflow_NWIS,gage,water_year
0,True,True,True,True,False,False,True,True,True
1,True,True,True,True,False,False,True,True,True
2,True,True,True,True,False,False,True,True,True
3,True,True,True,True,False,False,True,True,True
4,True,True,True,True,False,False,True,True,True
...,...,...,...,...,...,...,...,...,...
25928,True,False,True,False,False,True,True,True,True
25929,True,False,True,False,False,True,True,True,True
25930,True,False,True,False,False,True,True,True,True
25931,True,False,True,False,False,True,True,True,True


In [130]:
tt.isnull()

,Unnamed: 0,streamflow_VIC,time,streamflow_PRMS,streamflow_NWM2d0,streamflow_NWM2d1,streamflow_NWIS,gage,water_year
0,False,False,False,False,True,True,False,False,False
1,False,False,False,False,True,True,False,False,False
2,False,False,False,False,True,True,False,False,False
3,False,False,False,False,True,True,False,False,False
4,False,False,False,False,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...
25928,False,True,False,True,True,False,False,False,False
25929,False,True,False,True,True,False,False,False,False
25930,False,True,False,True,True,False,False,False,False
25931,False,True,False,True,True,False,False,False,False
